# Tutorial for HPS multiple A1-LCD WTARO 

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import time
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
#platform_name = 'CPU'
platform_name = 'CUDA'
#platform_name = 'OpenCL'

Parse a single A1LCD WT. The sequence length is 137.

In [2]:
#sequence = 'GSMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF' # WT
sequence = 'GSMASASSSQRGRSGSGNSGGGRGGGFGGNDNFGRGGNSSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNSGGGGSSNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYSAKPRNQGGYGGSSSSSSSGSGRRF' # ARO-
ca_pdb = 'init_A1LCD_WT_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [3]:
# insert molecules into the simulation box randomly
n_mol = 200
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])


Successfully inserted 200 molecules.


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [4]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
print('size of init_coord:', len(init_coord))
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 274*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)

# run simulation
start = time.time()
totaL_steps = 100000
protein.simulation.step(totaL_steps)
end = time.time()
print('Total time for %d steps: %.2f s' % (totaL_steps, end-start))

size of init_coord: 27400
Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,13482.504532247685,80349.35386801216,93831.85840025984,235.13768180997337,0
2000,20.000000000000327,10149.865337639694,85791.99172423956,95941.85706187926,251.06524297675963,2.97e+03
3000,30.00000000000189,9462.489429551917,87948.2779219809,97410.76735153282,257.37548834212515,2.87e+03
4000,40.00000000000061,9080.799842363895,89750.55444306399,98831.35428542788,262.6497451064566,3.1e+03
5000,49.99999999999862,8756.927297289181,90636.99823451655,99393.92553180573,265.2438709847977,3.21e+03
6000,59.99999999999663,8857.421672869383,91000.98508078586,99858.4

In [5]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=99)

Try to perform NPT simulations at the same temperature

In [ ]:
pressure = 1*unit.bar
protein.system.addForce(mm.MonteCarloBarostat(pressure, temperature))
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT_NPT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(200000)

Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,13571.563469442408,80484.13727999016,94055.70074943257,235.5321175771982,0
2000,20.000000000000327,10142.990802138465,86358.98203807301,96501.97284021147,252.7245069482224,2.92e+03
3000,30.00000000000189,9541.523276094755,89482.26032891106,99023.78360500581,261.8645980827629,2.54e+03
4000,40.00000000000061,8544.201126741094,92246.22711541016,100790.42824215126,269.9531851278405,2.48e+03
5000,49.99999999999862,8036.142701311247,93135.29946381995,101171.44216513119,272.55500332428596,2.43e+03
6000,59.99999999999663,8845.887642031128,92486.85103827469,101332.73868030582,270.6573569560711,2.33e+03
7000,69.9999999999989,9099.475983215034,92512.13448911824,101611.61047233327,270.7313474953015,2.28e+03
8000,80.00000000000402,9574.057935030527,92432.03127679709,102006.08921182761,270.4969301323235,